# Assignment 3 (Machine Learning)
> This assignment is solved by:-
* Name: Vishal Sehgal
* Group: CO11
* Roll Number: 102003265

### Question 1

> a

In [35]:
import numpy as np
import pandas as pd

In [36]:
df = pd.read_csv('./USA_Housing.csv')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
dtypes: float64(6)
memory usage: 234.5 KB


In [38]:
X = df.drop('Price', axis=1)
y = df['Price']

> b

In [39]:
# Making use of z-score standard scaling.
def scaler(data):
    sigma = np.std(data)
    mu = np.mean(data)
    for i in range(len(data)):
        data[i] = (data[i] - mu)/sigma
    return data

In [40]:
X['Avg. Area House Age'] = scaler(X['Avg. Area House Age'])

In [41]:
X['Avg. Area Income'] = scaler(X['Avg. Area Income'])

In [42]:
X['Avg. Area Number of Bedrooms'] = scaler(X['Avg. Area Number of Bedrooms'])

In [43]:
X['Avg. Area Number of Rooms'] = scaler(X['Avg. Area Number of Rooms'])

In [44]:
X['Area Population'] = scaler(X['Area Population'])

In [45]:
X.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,1.028660,-0.296927,0.021274,0.088062,-1.317599
1,1.000808,0.025902,-0.255506,-0.722301,0.403999
2,-0.684629,-0.112303,1.516243,0.930840,0.072410
3,-0.491499,1.221572,-1.393077,-0.584540,-0.186734
4,-0.807073,-0.944834,0.846742,0.201513,-0.988387


> c

In [66]:
def _5folds(X, y):
    fold_size = len(X)/5
    data_split = []
    for i in range(5):
        data_split.append(int(fold_size*i))
    X1 = X[data_split[0]:data_split[1]]
    X2 = X[data_split[1]:data_split[2]]
    X3 = X[data_split[2]:data_split[3]]
    X4 = X[data_split[3]:data_split[4]]
    X5 = X[data_split[4]:len(X)]
    y1 = y[data_split[0]:data_split[1]]
    y2 = y[data_split[1]:data_split[2]]
    y3 = y[data_split[2]:data_split[3]]
    y4 = y[data_split[3]:data_split[4]]
    y5 = y[data_split[4]:len(X)]
    return np.array([X1, X2, X3, X4, X5]), np.array([y1, y2, y3, y4, y5])

In [67]:
X_fold, y_fold = _5folds(X, y)

In [116]:
X_fold

array([[[ 1.02865969, -0.29692705,  0.02127433,  0.08806222,
         -1.31759867],
        [ 1.00080775,  0.02590164, -0.25550611, -0.72230146,
          0.40399945],
        [-0.68462915, -0.11230283,  1.5162435 ,  0.93084045,
          0.07240989],
        ...,
        [-0.97390997,  0.32742189,  0.3856136 ,  1.15774228,
          1.0579596 ],
        [ 1.84495107,  0.5761329 ,  1.76755667, -0.50350326,
         -1.04502559],
        [ 0.01449665, -1.48943947,  1.45395689,  1.78172231,
         -0.19616035]],

       [[-1.14104906, -1.31776484,  1.11567765, -0.77092328,
          0.40225336],
        [-0.17876139, -2.08742238,  0.50539953,  0.84170044,
         -1.32320392],
        [ 0.27530504,  1.94518516,  0.73935604,  0.84170044,
          0.41067075],
        ...,
        [ 0.0830251 , -1.09969937,  0.35582492,  0.09616586,
          1.96852391],
        [ 1.02483129,  0.53248907,  0.56305663,  0.09616586,
          0.65632178],
        [ 0.48858196,  0.64329388, -0.72409425, 

> d

In [141]:
y_fold.shape

(5, 1000)

In [157]:
beta_mat = []
for i in range(len(X_fold)):
    ind = np.array([])
    for j in range(len(X_fold)):
        if(i!=j):
            ind = np.append(ind, j)
    X_test = X_fold[i]
    y_test = y_fold[i]
    if(i!=0):
        X_train = np.array(X_fold[0][0:])
        y_train = np.array(y_fold[0])
    else:
        X_train = np.array(X_fold[1][0:])
        y_train = np.array(y_fold[1])
    for j in ind:
        if((i==0) and (j==1)):
            continue
        if(j==0):
            continue
        X_train = np.vstack([X_train, X_fold[int(j)][0:]])
    for j in ind:
        if((i==0) and (j==1)):
            continue
        if(j==0):
            continue
        y_train = np.append(y_train, y_fold[int(j)])
    X_train = np.insert(X_train, 0, values=1, axis=1)
    A = X_train.T.dot(X_train)
    B = np.linalg.inv(A)
    C = B.dot(X_train.T)
    beta = C.dot(y_train)
    beta_mat.append(beta)
    print(f"\nFor Cross validation {i+1}: \n\nThe beta matrix is: \n{beta}")
    X_test = np.insert(X_test, 0, values=1, axis=1)
    y_predict = X_test.dot(beta)
    error=y_test-y_predict
    square_error=np.power(error,2)
    sum_square_error=np.sum(square_error)
    mean_square_error=sum_square_error/len(y_predict)
    rms_error=np.sqrt(mean_square_error)
    y_mean=np.mean(y_test)
    total_variance=np.sum((y_test-y_mean)**2)
    print(f"\nR2-Score: {1-sum_square_error/total_variance}")


For Cross validation 1: 

The beta matrix is: 
[1231872.07704613  229923.70318635  164052.98579011  120923.57426579
    3533.77470973  151855.58542488]

R2-Score: 0.9175899480765107

For Cross validation 2: 

The beta matrix is: 
[1232586.40453402  228365.32287054  165201.14881927  122539.46855211
    1697.03433458  150055.81710562]

R2-Score: 0.920301549640113

For Cross validation 3: 

The beta matrix is: 
[1231448.09063372  230730.20810582  163554.55706976  121669.08713233
    2620.25215054  150923.12489298]

R2-Score: 0.9152429915320014

For Cross validation 4: 

The beta matrix is: 
[1.23144707e+06 2.29921558e+05 1.64523054e+05 1.19737507e+05
 1.12425659e+03 1.51317802e+05]

R2-Score: 0.9208503836977655

For Cross validation 5: 

The beta matrix is: 
[1233028.38536437  230778.59921985  163755.13607252  121885.1351222
    1247.01304382  150084.29759618]

R2-Score: 0.9138111758717495


> e

In [158]:
# We found that the best value for R2 is in 4th cross validation and hence having the best value of beta in it.
beta_mat[3]

array([1.23144707e+06, 2.29921558e+05, 1.64523054e+05, 1.19737507e+05,
       1.12425659e+03, 1.51317802e+05])

In [159]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [163]:
X_test = np.array(X_test)
X_test = np.insert(X_test, 0, values=1, axis=1)

In [164]:
y_predict = X_test.dot(beta_mat[3])

In [165]:
error=y_test-y_predict
square_error=np.power(error,2)
sum_square_error=np.sum(square_error)
mean_square_error=sum_square_error/len(y_predict)
rms_error=np.sqrt(mean_square_error)
y_mean=np.mean(y_test)
total_variance=np.sum((y_test-y_mean)**2)
print(f"\nR2-Score: {1-sum_square_error/total_variance}")


R2-Score: 0.9146868482147401


### Question 2